In [1]:
import pandas as pd
import datetime
import time
import unicodedata
import string
import requests
import mysql.connector
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from sshtunnel import SSHTunnelForwarder
import pymysql as db
import pandas as pd
import datetime
import sshtunnel
import os
# .env ファイルをロードして環境変数へ反映
from dotenv import load_dotenv
load_dotenv('.env')
# 環境変数を参照

def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 

def insert_data_bulk(df,cnx):
    insert_sql = f"""INSERT INTO {os.getenv('WORDPRESS_DB_TABLE')}(店舗名, 日付, Nのつく日, 都道府県, 機種名, 台番号, G数, 差枚, BB, RB,ART, BB確率, RB確率, ART確率, 合成確率) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    cnx.commit()
    print("Insert bulk data")

def delete_data(cnx,day):
    cursor = cnx.cursor()
    target_days_ago = datetime.date.today() - datetime.timedelta(days=day)
    target_days_ago_str = target_days_ago.strftime('%Y-%m-%d')
    target_days_ago_str
    sql = f"DELETE FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 日付 < '{target_days_ago_str} 00:00:00';"
    cursor.execute(sql)
    cnx.commit()

prefecture_list = ['鳥取県']#,'東京都''静岡県','岐阜県','愛知県','三重県'
line_token = os.getenv('LINE_TOKEN')
#print(line_token)
for prefecture in prefecture_list:
    try:
        post_line_text(f'{prefecture}MYSQL追加処理を開始します',line_token)
        cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']
        ichiran_all_tennpo_df = pd.DataFrame(index=[], columns=cols)
        yesterday = datetime.date.today() + datetime.timedelta(days=-2)
        url = f'https://{os.getenv("SCRAPING_DOMAIN")}/%E3%83%9B%E3%83%BC%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF/{prefecture}/'
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        table = soup.find_all('table')
        tenpo_ichiran_df =pd.read_html(str(table))[-1]
        print(tenpo_ichiran_df)
        all_parlar_count_number = str(len(tenpo_ichiran_df))
        post_line_text(f'{prefecture}は{all_parlar_count_number}の店舗が掲載されいます',line_token)
        count = 0
        error_count = 0
        for i, tenpo_name in enumerate(tenpo_ichiran_df['ホール名'] ):#tenpo_ichiran_df['ホール名']
            try:
                #time.sleep(1)
                print(i,tenpo_name)
                url = f'https://{os.getenv("SCRAPING_DOMAIN")}/{yesterday.strftime("%Y-%m-%d")}-{tenpo_name}'
                res = requests.get(url)
                soup = BeautifulSoup(res.text, 'html.parser')
                table = soup.find('table')
                dfs =pd.read_html(str(table))
                #display(tenpo_df)
                #time.sleep(1)
                for df in  dfs:
                    try:
                        if '機種名' in list(df.columns):
                            ichiran_df = df
                            ichiran_df['日付'] = yesterday.strftime('%Y-%m-%d')
                            ichiran_df['店舗名'] = tenpo_name
                            #print(tenpo_name)
                            ichiran_df['Nのつく日'] = yesterday.strftime('%d')[-1]
                            ichiran_df['都道府県'] = prefecture 
                            ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                            ichiran_all_tennpo_df =  pd.concat([ichiran_all_tennpo_df, ichiran_df])
                            print('成功',i,tenpo_name)
                            break
                        else:
                            print('見つかりませんでした',i,tenpo_name)
                        count += 1
                    except Exception as e:
                        print(tenpo_name,e)
                        error_count += 1
                        #time.sleep(1)
                        continue
            except Exception as e:
                print(tenpo_name,e)
                continue

            # if i > 0:
            #     break
        cols = ichiran_all_tennpo_df.columns.tolist()
        cols = cols[-4:] + cols[:-4]
        ichiran_all_tennpo_df = ichiran_all_tennpo_df[cols]  #    OR    df = df.ix[:, cols]
        ichiran_all_tennpo_df['ART']= ichiran_all_tennpo_df['ART'].fillna(0)
        ichiran_all_tennpo_df['BB']= ichiran_all_tennpo_df['BB'].fillna(0)
        ichiran_all_tennpo_df['RB']= ichiran_all_tennpo_df['RB'].fillna(0)
        ichiran_all_tennpo_df['差枚']= ichiran_all_tennpo_df['差枚'].fillna(0)
        ichiran_all_tennpo_df['G数']= ichiran_all_tennpo_df['G数'].fillna(0)
        ichiran_all_tennpo_df = ichiran_all_tennpo_df.fillna('')
        #print(ichiran_all_tennpo_df.iloc[:5])
        # SSH 接続 踏み台接続
        with sshtunnel.SSHTunnelForwarder(
            (os.getenv('SSH_USERNAME'), 10022), 
            ssh_username="pachislot777", 
            ssh_private_key_password=os.getenv('SSH_PRIVATE_KEY_PASSWORD'), 
            ssh_pkey=r"sercret\akasaka.key", 
            remote_bind_address=("mysql8055.xserver.jp", 3306 )
            ) as server:

            # SSH接続確認
            print(f"local bind port: {server.local_bind_port}")
            # データベース接続
            cnx = mysql.connector.connect(
                host="localhost", 
                port=server.local_bind_port, 
                user=os.getenv('WORDPRESS_DB_ID'), 
                password=os.getenv('DB_PASSWORD'), 
                database=os.getenv('WORDPRESS_DB_NAME'), 
                charset="utf8",
                use_pure=True
                )

            # 接続確認
            print(f"sql connection status: {cnx.is_connected()}")
            cursor = cnx.cursor()
            #insert_data_bulk(ichiran_all_tennpo_df,cnx)
            tenpo_name_number = len(ichiran_all_tennpo_df['店舗名'].unique())
            post_line_text(f'{prefecture} {tenpo_name_number}/{all_parlar_count_number}件のSQL追加処理に成功しました',line_token)
            delete_data(cnx,35)
            # 終了
            cnx.close()

    except Exception as e:
        print(e)
        post_line_text(f'{prefecture}処理失敗{e}',line_token)

    finally:
        print('終了')


              ホール名  市区郡
0            UFO倉吉  倉吉市
1             UFO叶  鳥取市
2            UFO吉方  鳥取市
3            UFO安長  鳥取市
4            UFO秋里  鳥取市
5          アルファ境港店  境港市
6         アンコールセブン  米子市
7    グランワールドカップ米子店  米子市
8    グランワールドカップ鳥取店  鳥取市
9      クレイジーチャンス岸本  西伯郡
10  ジャンボマックス888鳥取店  鳥取市
11     ジャンボマックス米子店  米子市
12        デルパラ7境港店  境港市
13        デルパラ7米原店  米子市
14         マルハン鳥取店  鳥取市
15         マンモス倉吉店  東伯郡
16         マンモス桜谷店  鳥取市
17         マンモス湖山店  鳥取市
18      ワールドカップ鳥取店  鳥取市
0 UFO倉吉
成功 0 UFO倉吉
1 UFO叶
見つかりませんでした 1 UFO叶
2 UFO吉方
成功 2 UFO吉方
3 UFO安長
見つかりませんでした 3 UFO安長
4 UFO秋里
見つかりませんでした 4 UFO秋里
5 アルファ境港店
終了


KeyboardInterrupt: 

In [12]:
for i, tenpo_name in enumerate(['UFO叶'] ):#tenpo_ichiran_df['ホール名']
            try:
                #time.sleep(1)
                print(i,tenpo_name)
                url = f'https://{os.getenv("SCRAPING_DOMAIN")}/{yesterday.strftime("%Y-%m-%d")}-{tenpo_name}-data'
                res = requests.get(url)
                soup = BeautifulSoup(res.text, 'html.parser')
                table = soup.find(id = "all_data_table")
                dfs =pd.read_html(str(table))
                #display(tenpo_df)
                #time.sleep(1)
                for df in  dfs:
                    try:
                        if '機種名' in list(df.columns):
                            ichiran_df = df
                            ichiran_df['日付'] = yesterday.strftime('%Y-%m-%d')
                            ichiran_df['店舗名'] = tenpo_name
                            #print(tenpo_name)
                            ichiran_df['Nのつく日'] = yesterday.strftime('%d')[-1]
                            ichiran_df['都道府県'] = prefecture 
                            ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                            ichiran_all_tennpo_df =  pd.concat([ichiran_all_tennpo_df, ichiran_df])
                            print('成功',i,tenpo_name)
                            break
                        else:
                            print('見つかりませんでした',i,tenpo_name)
                        count += 1
                    except Exception as e:
                        print(tenpo_name,e)
                        error_count += 1
                        #time.sleep(1)
                        continue
            except Exception as e:
                print(tenpo_name,e)
                continue

0 UFO叶
成功 0 UFO叶


In [11]:
soup.find_all('table')[1]

<table class="fixed_get_medals_table" id="all_data_table">
<thead>
<tr>
<th class="fixed01">機種名</th>
<th>台番号</th>
<th>G数</th>
<th>差枚</th>
<th>BB</th>
<th>RB</th>
<th>合成確率</th>
<th>BB確率</th>
<th>RB確率</th>
</tr>
</thead>
<tbody>
<tr>
<td class="fixed01" style="font-size:14px;line-height:150%;">アイムジャグラーEX-TP</td>
<td class="table_cells">379</td>
<td class="table_cells">5,510</td>
<td class="table_cells">-1,845</td>
<td class="table_cells">13</td>
<td class="table_cells">20</td>
<td class="table_cells">1/167.0</td>
<td class="table_cells">1/423.8</td>
<td class="table_cells">1/275.5</td>
</tr>
<tr>
<td class="fixed01" style="font-size:14px;line-height:150%;">アイムジャグラーEX-TP</td>
<td class="table_cells">380</td>
<td class="table_cells">7,704</td>
<td class="table_cells">-220</td>
<td class="table_cells">26</td>
<td class="table_cells">32</td>
<td class="table_cells">1/132.8</td>
<td class="table_cells">1/296.3</td>
<td class="table_cells">1/240.8</td>
</tr>
<tr>
<td class="fixed01" style="fon

In [8]:
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]>
<html class="ie6" dir="ltr" lang="ja"
	prefix="og: https://ogp.me/ns#" > <![endif]-->
<!--[if IE 7]>
<html class="i7" dir="ltr" lang="ja"
	prefix="og: https://ogp.me/ns#" > <![endif]-->
<!--[if IE 8]>
<html class="ie" dir="ltr" lang="ja"
	prefix="og: https://ogp.me/ns#" > <![endif]-->
<!--[if gt IE 8]><!-->
<html class="" dir="ltr" lang="ja" prefix="og: https://ogp.me/ns#">
<!--<![endif]-->
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,user-scalable=no,viewport-fit=cover" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://ana-slo.com/feed/" rel="alternate" title="アナスロ RSS Feed" type="application/rss+xml"/>
<link href="https://ana-slo.com/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
		<script src="https://ana-slo.com/wp-content/themes/affinger5/js/html5shiv.js"></script>
		<![endif]-

In [1]:
import pandas as pd
import datetime
import time
import unicodedata
import string
import requests
import mysql.connector
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from sshtunnel import SSHTunnelForwarder
import pymysql as db
import pandas as pd
import datetime
import sshtunnel
import os
# .env ファイルをロードして環境変数へ反映
from dotenv import load_dotenv
load_dotenv('.env')
# 環境変数を参照

def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 

def insert_data_bulk(df,cnx):
    insert_sql = f"""INSERT INTO {os.getenv('WORDPRESS_DB_TABLE')}(店舗名, 日付, Nのつく日, 都道府県, 機種名, 台番号, G数, 差枚, BB, RB,ART, BB確率, RB確率, ART確率, 合成確率) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    cnx.commit()
    print("Insert bulk data")

def delete_data(cnx,day):
    cursor = cnx.cursor()
    target_days_ago = datetime.date.today() - datetime.timedelta(days=day)
    target_days_ago_str = target_days_ago.strftime('%Y-%m-%d')
    target_days_ago_str
    sql = f"DELETE FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 日付 < '{target_days_ago_str} 00:00:00';"
    cursor.execute(sql)
    cnx.commit()


In [24]:


prefecture_list = ['愛知県']#,'東京都'
line_token = os.getenv('LINE_TOKEN')
#print(line_token)
for prefecture in prefecture_list:
    post_line_text(f'{prefecture}MYSQL追加処理を開始します',line_token)
    cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']
    ichiran_all_tennpo_df = pd.DataFrame(index=[], columns=cols)
    yesterday = datetime.date.today() + datetime.timedelta(days=-1)
    url = f'https://{os.getenv("SCRAPING_DOMAIN")}/%E3%83%9B%E3%83%BC%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF/{prefecture}/'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.find_all('table')
    tenpo_ichiran_df =pd.read_html(str(table))[-1]
    print(tenpo_ichiran_df)
    i = 0
    for tenpo_name in tenpo_ichiran_df['ホール名'] :#tenpo_ichiran_df['ホール名']
        #print(tenpo_name)
        url = f'https://{os.getenv("SCRAPING_DOMAIN")}/{yesterday.strftime("%Y-%m-%d")}-{tenpo_name}'
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        table = soup.find('table')
        dfs =pd.read_html(str(table))
        #display(tenpo_df)
        time.sleep(1)
        for df in  dfs:
        #print(df.columns)
            if '機種名' in list(df.columns):
                ichiran_df = df
                ichiran_df['日付'] = yesterday.strftime('%Y-%m-%d')
                ichiran_df['店舗名'] = tenpo_name
                print(tenpo_name)
                ichiran_df['Nのつく日'] = yesterday.strftime('%d')[-1]
                ichiran_df['都道府県'] = prefecture 
                ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                ichiran_all_tennpo_df =  pd.concat([ichiran_all_tennpo_df, ichiran_df])
                break
        break

          ホール名      市区郡
0     100℃大曽根店   名古屋市東区
1      100℃守山店  名古屋市守山区
2       ABC豊川店      豊川市
3    APAN21丸根店      豊田市
4    APAN21勝川店     春日井市
..         ...      ...
275   東宝プラザ城西店   名古屋市西区
276   東宝プラザ師勝店    北名古屋市
277   東宝プラザ野並店  名古屋市天白区
278       稲口会館     春日井市
279      豊川コロナ      豊川市

[280 rows x 2 columns]
100℃大曽根店


In [22]:
url = f'https://{os.getenv("SCRAPING_DOMAIN")}/{yesterday.strftime("%Y-%m-%d")}-ABC豊川店'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
table = soup.find('table')
dfs =pd.read_html(str(table))
dfs[0]

,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率
0,マイジャグラーV,321,6495,-553,22,22,0,1/147.6,1/295.2,1/295.2,1/0.0
1,マイジャグラーV,322,4078,129,18,8,0,1/156.8,1/226.6,1/509.8,1/0.0
2,マイジャグラーV,323,4194,-1262,12,8,0,1/209.7,1/349.5,1/524.2,1/0.0
3,マイジャグラーV,324,3791,-668,12,8,0,1/189.6,1/315.9,1/473.9,1/0.0
4,マイジャグラーV,325,4312,-1178,11,14,0,1/172.5,1/392.0,1/308.0,1/0.0
...,...,...,...,...,...,...,...,...,...,...,...
288,閃乱カグラ BURST UP,433,1890,-262,0,12,11,1/82.2,1/0.0,1/157.5,1/171.8
289,この素晴らしい世界に祝福を！,434,6082,-168,0,17,82,1/61.4,1/0.0,1/357.8,1/74.2
290,ハードボイルド,436,3534,1374,14,10,15,1/90.6,1/252.4,1/353.4,1/235.6
291,真俺の空,448,420,285,0,1,6,1/60.0,1/0.0,1/420.0,1/70.0


,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率
0,マイジャグラーV,321,6495,-553,22,22,0,1/147.6,1/295.2,1/295.2,1/0.0
1,マイジャグラーV,322,4078,129,18,8,0,1/156.8,1/226.6,1/509.8,1/0.0
2,マイジャグラーV,323,4194,-1262,12,8,0,1/209.7,1/349.5,1/524.2,1/0.0
3,マイジャグラーV,324,3791,-668,12,8,0,1/189.6,1/315.9,1/473.9,1/0.0
4,マイジャグラーV,325,4312,-1178,11,14,0,1/172.5,1/392.0,1/308.0,1/0.0
...,...,...,...,...,...,...,...,...,...,...,...
288,閃乱カグラ BURST UP,433,1890,-262,0,12,11,1/82.2,1/0.0,1/157.5,1/171.8
289,この素晴らしい世界に祝福を！,434,6082,-168,0,17,82,1/61.4,1/0.0,1/357.8,1/74.2
290,ハードボイルド,436,3534,1374,14,10,15,1/90.6,1/252.4,1/353.4,1/235.6
291,真俺の空,448,420,285,0,1,6,1/60.0,1/0.0,1/420.0,1/70.0


In [12]:
pd.read_html(table)

TypeError: 'NoneType' object is not callable

In [8]:

dfs = pd.read_html(res.text)

In [6]:
dfs[0]

,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率
0,マイジャグラーV,321,6495,-553,22,22,0,1/147.6,1/295.2,1/295.2,1/0.0
1,マイジャグラーV,322,4078,129,18,8,0,1/156.8,1/226.6,1/509.8,1/0.0
2,マイジャグラーV,323,4194,-1262,12,8,0,1/209.7,1/349.5,1/524.2,1/0.0
3,マイジャグラーV,324,3791,-668,12,8,0,1/189.6,1/315.9,1/473.9,1/0.0
4,マイジャグラーV,325,4312,-1178,11,14,0,1/172.5,1/392.0,1/308.0,1/0.0
...,...,...,...,...,...,...,...,...,...,...,...
288,閃乱カグラ BURST UP,433,1890,-262,0,12,11,1/82.2,1/0.0,1/157.5,1/171.8
289,この素晴らしい世界に祝福を！,434,6082,-168,0,17,82,1/61.4,1/0.0,1/357.8,1/74.2
290,ハードボイルド,436,3534,1374,14,10,15,1/90.6,1/252.4,1/353.4,1/235.6
291,真俺の空,448,420,285,0,1,6,1/60.0,1/0.0,1/420.0,1/70.0


In [ ]:

options = Options()
options.add_argument("--headless")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-gpu")
# options.add_argument("--disable-features=NetworkService")
# options.add_argument("--window-size=1920x1080")
# options.add_argument("--disable-features=VizDisplayCompositor")
# UA = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
# options.add_argument('--user-agent=' + UA)
# mobile_emulation = { "deviceName": "Galaxy S5" }
# options.add_experimental_option("mobileEmulation", mobile_emulation)

prefecture_list = ['愛知県']#,'東京都'

line_token = os.getenv('LINE_TOKEN')
#print(line_token)
for prefecture in prefecture_list:
    try:
        post_line_text(f'{prefecture}MYSQL追加処理を開始します',line_token)
        cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']
        ichiran_all_tennpo_df = pd.DataFrame(index=[], columns=cols)
        yesterday = datetime.date.today() + datetime.timedelta(days=-1)
        browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)
        url = f'https://{os.getenv("SCRAPING_DOMAIN")}/%E3%83%9B%E3%83%BC%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF/{prefecture}/'
        browser.get(url)
        html = browser.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        tenpo_ichiran_df = pd.read_html(html)[-1]
        #print(tenpo_ichiran_df['ホール名'])

        i = 0
        for tenpo_name in tenpo_ichiran_df['ホール名'] :#tenpo_ichiran_df['ホール名']
            try:
                #print(tenpo_name)
                url = f'https://{os.getenv("SCRAPING_DOMAIN")}/{yesterday.strftime("%Y-%m-%d")}-{tenpo_name}'
                browser.get(url)
                html = browser.page_source.encode('utf-8')
                dfs = pd.read_html(html)
                #display(tenpo_df)
                time.sleep(1)
                for df in  dfs:
                #print(df.columns)
                    if '機種名' in list(df.columns):
                        ichiran_df = df
                        ichiran_df['日付'] = yesterday.strftime('%Y-%m-%d')
                        ichiran_df['店舗名'] = tenpo_name
                        print(tenpo_name)
                        ichiran_df['Nのつく日'] = yesterday.strftime('%d')[-1]
                        ichiran_df['都道府県'] = prefecture 
                        ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                        ichiran_all_tennpo_df =  pd.concat([ichiran_all_tennpo_df, ichiran_df])
                        break
            except:
                time.sleep(1)
                continue
            i += 1
            # if i > 0:
            #     break
        browser.quit()
        cols = ichiran_all_tennpo_df.columns.tolist()
        cols = cols[-4:] + cols[:-4]
        ichiran_all_tennpo_df = ichiran_all_tennpo_df[cols]  #    OR    df = df.ix[:, cols]
        ichiran_all_tennpo_df['ART']= ichiran_all_tennpo_df['ART'].fillna(0)
        ichiran_all_tennpo_df['BB']= ichiran_all_tennpo_df['BB'].fillna(0)
        ichiran_all_tennpo_df['RB']= ichiran_all_tennpo_df['RB'].fillna(0)
        ichiran_all_tennpo_df['差枚']= ichiran_all_tennpo_df['差枚'].fillna(0)
        ichiran_all_tennpo_df['G数']= ichiran_all_tennpo_df['G数'].fillna(0)
        ichiran_all_tennpo_df = ichiran_all_tennpo_df.fillna('')
        #print(ichiran_all_tennpo_df.iloc[:5])
        # SSH 接続 踏み台接続
        with sshtunnel.SSHTunnelForwarder(
            (os.getenv('SSH_USERNAME'), 10022), 
            ssh_username="pachislot777", 
            ssh_private_key_password=os.getenv('SSH_PRIVATE_KEY_PASSWORD'), 
            ssh_pkey=r"sercret\akasaka.key", 
            remote_bind_address=("mysql8055.xserver.jp", 3306 )
            ) as server:

            # SSH接続確認
            print(f"local bind port: {server.local_bind_port}")
            # データベース接続
            cnx = mysql.connector.connect(
                host="localhost", 
                port=server.local_bind_port, 
                user=os.getenv('WORDPRESS_DB_ID'), 
                password=os.getenv('DB_PASSWORD'), 
                database=os.getenv('WORDPRESS_DB_NAME'), 
                charset="utf8",
                use_pure=True
                )

            # 接続確認
            print(f"sql connection status: {cnx.is_connected()}")
            cursor = cnx.cursor()
            insert_data_bulk(ichiran_all_tennpo_df,cnx)
            tenpo_name_number = len(ichiran_all_tennpo_df['店舗名'].unique())
            post_line_text(f'{prefecture} {tenpo_name_number}件のSQL追加処理に成功しました',line_token)
            delete_data(cnx,35)
            # 終了
            cnx.close()

    except Exception as e:
        print(e)
        post_line_text(f'{prefecture}処理失敗{e}',line_token)

    finally:
        print('終了')
